<a href="https://colab.research.google.com/github/kalind789/PyTorch_Vision_Learning/blob/main/BetterCNNImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# For dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

In [22]:
torch.manual_seed(42)

In [23]:
torch.cuda.is_available()

True

In [24]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [25]:
transform = transforms.ToTensor()
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

In [26]:
batch_size = 8

trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2)

In [27]:
train_iter = iter(trainloader)
images, labels = next(train_iter)
images.shape, images.dtype, labels.shape

(torch.Size([8, 3, 32, 32]), torch.float32, torch.Size([8]))

In [28]:
class MyNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.pool = nn.MaxPool2d(2, 2)

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3)
    self.conv2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
    self.conv3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3)

    self.fc1 = nn.Linear(512*2*2, 1024)
    self.drop = nn.Dropout(p=0.3)
    self.fc2 = nn.Linear(1024, 1024)
    self.fc3 = nn.Linear(1024, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))

    x = torch.flatten(x, 1)

    x = F.relu(self.fc1(x))
    x = self.drop(x)
    x = F.relu(self.fc2(x))
    x = self.drop(x)
    x = self.fc3(x)

    return x

In [29]:
net = MyNet().to(device)

In [32]:
optimizer = optim.Adam(net.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()

In [33]:
for epoch in range(5):
  print(f"Epoch: {epoch+1}")
  running_loss = 0.0
  running_correct = 0
  running_total = 0
  for idx, data in enumerate(trainloader, 0):
    images, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs = net(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    running_total += labels.size(0)
    running_correct += (predicted == labels).sum().item()

    if idx % 1000 == 999:
      print(f'[{epoch + 1}, {idx + 1:5d}] loss: {running_loss / 1000:.3f}')
      avg_acc_across_batches = 100 * running_correct / running_total
      print(f'Average accuracy across batches: {avg_acc_across_batches:.2f} %')
      running_loss = 0.0
      running_correct = 0
      running_total = 0

  print(f"-------------------------------")

Epoch: 1
[1,  1000] loss: 1.223
Average accuracy across batches: 55.54 %
[1,  2000] loss: 1.177
Average accuracy across batches: 57.55 %
[1,  3000] loss: 1.169
Average accuracy across batches: 58.41 %
[1,  4000] loss: 1.151
Average accuracy across batches: 58.99 %
[1,  5000] loss: 1.142
Average accuracy across batches: 59.17 %
[1,  6000] loss: 1.108
Average accuracy across batches: 60.21 %
-------------------------------
Epoch: 2
[2,  1000] loss: 1.071
Average accuracy across batches: 61.66 %
[2,  2000] loss: 1.085
Average accuracy across batches: 61.21 %
[2,  3000] loss: 1.054
Average accuracy across batches: 62.48 %
[2,  4000] loss: 1.058
Average accuracy across batches: 62.79 %
[2,  5000] loss: 1.047
Average accuracy across batches: 63.31 %
[2,  6000] loss: 1.037
Average accuracy across batches: 63.16 %
-------------------------------
Epoch: 3
[3,  1000] loss: 1.008
Average accuracy across batches: 64.09 %
[3,  2000] loss: 0.989
Average accuracy across batches: 64.97 %
[3,  3000] lo

In [34]:
total = 0.0
correct = 0.0
with torch.no_grad():
  for data in testloader:
      images, labels = data[0].to(device), data[1].to(device)
      # calculate outputs by running images through the network
      outputs = net(images)
      # the class with the highest energy is what we choose as prediction
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 65.0 %
